<a href="https://colab.research.google.com/github/yskim0/study/blob/master/Keras_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Keras Guide


* Sequential 모델

케라스에서는 층(layer)을 조합하여 모델을 만듦.

`모델은 층의 그래프`임

`tf.keras.Sequenetial`

& Sequential보다 더 고수준의 모델을 구성하는 방법을 배우려면 다음을 참고하세요:

[케라스 함수형 API 가이드 ](https://www.tensorflow.org/guide/keras/functional)

[클래스 상속을 통하여 층과 모델을 밑바닥부터 만드는 방법](https://www.tensorflow.org/guide/keras/custom_layers_and_models)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 또 하나를 추가합니다:
model.add(layers.Dense(64, activation='relu'))
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
model.add(layers.Dense(10, activation='softmax'))

* 층 설정


`activation` 

층의 활성화 함수를 설정합니다.

이 매개변수에는 기본으로 제공되는 함수의 이름을 쓰거나 호출 가능한 객체를 지정할 수 있습니다.

기본값은 활성화 함수를 적용하지 않는 것입니다.

`kernel_initializer와 bias_initializer`

층의 가중치(weight)(커널(kernel)과 절편(bias))를 초기화하는 방법입니다. 

내장 함수나 호출 가능한 객체를 지정합니다. 

기본값은 "glorot_uniform" 초기화입니다.


`kernel_regularizer와 bias_regularizer`

L1 또는 L2 규제(regularization)와 같이 층의 가중치(커널과 절편)에 적용할 규제 방법을 지정합니다. 기본값은 규제를 적용하지 않는 것입니다.

In [2]:
# 시그모이드 활성화 층을 만듭니다:
layers.Dense(64, activation='sigmoid')
# 또는 다음도 가능합니다:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# 커널 행렬에 L1 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# 절편 벡터에 L2 규제가 적용된 선형 활성화 층. 하이퍼파라미터 0.01은 규제의 양을 조절합니다:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# 커널을 랜덤한 직교 행렬로 초기화한 선형 활성화 층:
layers.Dense(64, kernel_initializer='orthogonal')

# 절편 벡터를 상수 2.0으로 설정한 선형 활성화 층:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

## 훈련과 평가

* `tf.keras.Model.compile` (3가지 매개 변수)



1.   `optimizer`
  훈련 과정을 설정함. 
  
  기본 매개변수를 사용할 경우 'adam'이나 'sgd'와 같이 문자열로 지정할 수도 있습니다.

2.   `loss`
  최적화 과정에서 최소화될 손실 함수(loss function)를 설정

  평균 제곱 오차(mse)와 categorical_crossentropy, binary_crossentropy 등이 자주 사용

  손실 함수의 이름을 지정하거나 tf.keras.losses 모듈 아래의 호출 가능한 객체를 전달 가능

3.    `metrics`
  훈련을 모니터링하기 위해 사용됨.

4.    추가적으로 모델의 훈련과 평가를 즉시 실행하려면 run_eagerly=True 매개변수를 전달하면 됨.




```
# 평균 제곱 오차로 회귀 모델을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # 평균 제곱 오차
              metrics=['mae'])  # 평균 절댓값 오차

# 크로스엔트로피 손실 함수로 분류 모델을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
```





In [3]:
model = tf.keras.Sequential([
# 64개의 유닛을 가진 완전 연결 층을 모델에 추가합니다:
layers.Dense(64, activation='relu', input_shape=(32,)),
# 또 하나를 추가합니다:
layers.Dense(64, activation='relu'),
# 10개의 출력 유닛을 가진 소프트맥스 층을 추가합니다:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


* `tf.keras.Model.fit` (3가지 매개 변수)

1.  `epochs`

훈련은 에포크로 구성됨.
 
한 에포크는 전체 입력 데이터를 한번 순회하는 것 (작은 배치로 나누어 수행)

2.  `batch size`

넘파이 데이터를 전달하면 **모델은 데이터를 작은 배치로 나누고 훈련 과정에서 이 배치를 순회**

이 정수 값은 배치의 크기를 지정. 

(전체 샘플 개수가 배치 크기로 나누어 떨어지지 않으면 마지막 배치의 크기는 더 작을 수 있음.)

3.  `validation_data`

모델의 프로토타입(prototype)을 만들 때는 **검증 데이터(validation data)**에서 간편하게 성능을 모니터링해야 합니다. 

입력과 레이블(label)의 튜플을 이 매개변수로 전달하면 

에포크가 끝날 때마다 추론 모드(inference mode)에서 **전달된 데이터의 손실과 측정 지표를 출력**합니다.


---
---


> 넘파이 데이터를 사용한 훈련

데이터셋이 작은 경우 넘파이(NumPy) 배열을 메모리에 적재하여 모델을 훈련하고 평가합니다. 

모델은 fit 메서드를 통해서 훈련 데이터를 학습합니다:

In [4]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 385us/sample - loss: 12.2870 - acc: 0.0960
Epoch 2/10
1000/1000 [==============================] - 0s 52us/sample - loss: 17.9181 - acc: 0.0980
Epoch 3/10
1000/1000 [==============================] - 0s 63us/sample - loss: 33.7644 - acc: 0.1000
Epoch 4/10
1000/1000 [==============================] - 0s 61us/sample - loss: 64.3578 - acc: 0.1060
Epoch 5/10
1000/1000 [==============================] - 0s 60us/sample - loss: 111.6877 - acc: 0.1040
Epoch 6/10
1000/1000 [==============================] - 0s 64us/sample - loss: 166.5614 - acc: 0.1040
Epoch 7/10
1000/1000 [==============================] - 0s 56us/sample - loss: 231.0664 - acc: 0.1070
Epoch 8/10
1000/1000 [==============================] - 0s 60us/sample - loss: 299.3644 - acc: 0.1110
Epoch 9/10
1000/1000 [==============================] - 0s 53us/sample - loss: 346.8318 - acc: 0.1070
Epoch 10/10
1000/1000 [==============================] - 0s 63u

In [5]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 91us/sample - loss: 389.4693 - acc: 0.1000 - val_loss: 411.2200 - val_acc: 0.1200
Epoch 2/10
1000/1000 [==============================] - 0s 55us/sample - loss: 367.7762 - acc: 0.1090 - val_loss: 371.8433 - val_acc: 0.1200
Epoch 3/10
1000/1000 [==============================] - 0s 52us/sample - loss: 327.9860 - acc: 0.0990 - val_loss: 291.8529 - val_acc: 0.1800
Epoch 4/10
1000/1000 [==============================] - 0s 57us/sample - loss: 239.2887 - acc: 0.0920 - val_loss: 211.6064 - val_acc: 0.1200
Epoch 5/10
1000/1000 [==============================] - 0s 65us/sample - loss: 146.4238 - acc: 0.0940 - val_loss: 97.5406 - val_acc: 0.1200
Epoch 6/10
1000/1000 [==============================] - 0s 56us/sample - loss: 77.2906 - acc: 0.1110 - val_loss: 56.6985 - val_acc: 0.1200
Epoch 7/10
1000/1000 [==============================] - 0s 56us/sample - loss: 85.1002 - acc: 0.1120 - val_los

*  tf.data 데이터셋을 사용한 훈련

데이터셋 API를 사용하여 대규모 데이터셋이나 복수의 장치로 확장시킬 수 있습니다. 

fit 메서드에 tf.data.Dataset 객체를 전달

**여기에서 fit 메서드는 steps_per_epoch 매개변수를 사용합니다. 
다음 에포크로 넘어가기 전에 모델이 수행할 훈련 단계 횟수입니다. Dataset이 배치 데이터를 생성하기 때문에 batch_size가 필요하지 않습니다.**

Dataset은 검증 데이터에도 사용 가능


In [6]:
# 예제 `Dataset` 객체를 만듭니다:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

# Dataset에서 `fit` 메서드를 호출할 때 `steps_per_epoch` 설정을 잊지 마세요.
model.fit(dataset, epochs=10, steps_per_epoch=30)

Train on 30 steps
Epoch 1/10
30/30 [==============================] - 0s 4ms/step - loss: 129.6700 - acc: 0.0990
Epoch 2/10
 1/30 [>.............................] - ETA: 0s - loss: 202.1968 - acc: 0.0938WARNING:tensorflow:Your dataset ran out of data; interrupting training. Make sure that your dataset can generate at least `steps_per_epoch * epochs` batches (in this case, 300 batches). You may need to use the repeat() function when building your dataset.


In [7]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32)

model.fit(dataset, epochs=10,
          validation_data=val_dataset)

Train on 32 steps, validate on 4 steps
Epoch 1/10
32/32 [==============================] - 0s 5ms/step - loss: 179.3362 - acc: 0.0930 - val_loss: 188.4568 - val_acc: 0.0700
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 170.0612 - acc: 0.1100 - val_loss: 160.2992 - val_acc: 0.1200
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 190.4937 - acc: 0.0780 - val_loss: 209.3025 - val_acc: 0.1500
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 284.2324 - acc: 0.1010 - val_loss: 213.4948 - val_acc: 0.1100
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 285.1647 - acc: 0.1240 - val_loss: 272.3232 - val_acc: 0.0900
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 289.6243 - acc: 0.0990 - val_loss: 362.7060 - val_acc: 0.0900
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 327.4941 - acc: 0.1050 - val_loss: 311.5286 - val_acc: 0.0600
Epoch 8/10
32/32 [=====

##평가와 예측

`tf.keras.Model.evaluate`와 `tf.keras.Model.predict` 메서드에는 

넘파이 배열이나 tf.data.Dataset을 사용할 수 있습니다.

주어진 데이터로 추론 모드의 손실이나 지표를 평가합니다:

In [8]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

model.evaluate(dataset, steps=30)

30/30 [==============================] - 0s 2ms/step - loss: 434.3732 - acc: 0.1083


[434.37316080729164, 0.108333334]

In [9]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


## 고급 모델 만들기



### 함수형 API

In [10]:
inputs = tf.keras.Input(shape=(32,))  # 입력 플레이스홀더를 반환합니다.

# 층 객체는 텐서를 사용하여 호출되고 텐서를 반환합니다.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=predictions)

# 컴파일 단계는 훈련 과정을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5번의 에포크 동안 훈련합니다.
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 116us/sample - loss: 12.5132 - acc: 0.1030
Epoch 2/5
1000/1000 [==============================] - 0s 42us/sample - loss: 18.0297 - acc: 0.0980
Epoch 3/5
1000/1000 [==============================] - 0s 45us/sample - loss: 29.4902 - acc: 0.1030
Epoch 4/5
1000/1000 [==============================] - 0s 41us/sample - loss: 45.0870 - acc: 0.1080
Epoch 5/5
1000/1000 [==============================] - 0s 42us/sample - loss: 63.5550 - acc: 0.1090


###모델 클라스 상속

tf.keras.Model 클래스를 상속하고 자신만의 정방향 패스(forward pass)을 정의하여 완전히 커스터마이징된 모델을 만들 수 있습니다. 

__init__ 메서드에서 층을 만들어 클래스 객체의 속성으로 지정합니다. 정방향 패스는 call 메서드에 정의합니다.

즉시 실행이 활성화되어 있을 때 정방향 패스를 명령형 프로그래밍 방식으로 작성할 수 있기 때문에 모델 클래스 상속이 매우 유용합니다.

*노트*: 정방향 패스를 항상 명령형 프로그래밍 방식으로 실행하려면 super 객체의 생성자를 호출할 때 dynamic=True를 지정하세요.

중요 포인트: 작업에 맞는 API를 사용하세요. 모델 클래스 상속은 유연성을 제공하지만 복잡도가 증가하고 사용자 오류가 발생할 가능성이 높아집니다. 가능한한 함수형 API를 사용하세요.

In [0]:
class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # 층을 정의합니다.
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # 정방향 패스를 정의합니다.
    # `__init__` 메서드에서 정의한 층을 사용합니다.
    x = self.dense_1(inputs)
    return self.dense_2(x)

In [12]:
model = MyModel(num_classes=10)

# 컴파일 단계는 훈련 과정을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5번의 에포크 동안 훈련합니다.
model.fit(data, labels, batch_size=32, epochs=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 212us/sample - loss: 11.4770 - acc: 0.1100
Epoch 2/5
1000/1000 [==============================] - 0s 45us/sample - loss: 11.4546 - acc: 0.1210
Epoch 3/5
1000/1000 [==============================] - 0s 41us/sample - loss: 11.4498 - acc: 0.1070
Epoch 4/5
1000/1000 [==============================] - 0s 50us/sample - loss: 11.4472 - acc: 0.1070
Epoch 5/5
1000/1000 [==============================] - 0s 43us/sample - loss: 11.4456 - acc: 0.1020


### 맞춤형 층

 `tf.keras.layers.Layer` 클래스를 상속하고 다음 메서드를 구현

* `__init__`: 이 층에서 사용되는 하위 층을 정의할 수 있습니다.
* `build`: 층의 가중치를 만듭니다. add_weight 메서드를 사용해 가중치를 추가합니다.
* `call`: 정방향 패스를 구현합니다.

In [0]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    # 이 층에서 훈련할 가중치 변수를 만듭니다.
    self.kernel = self.add_weight(name='kernel',
                                  shape=(input_shape[1], self.output_dim),
                                  initializer='uniform',
                                  trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [14]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# 컴파일 단계는 훈련 과정을 설정합니다.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5번의 에포크 동안 훈련합니다.
model.fit(data, labels, batch_size=32, epochs=5)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 85us/sample - loss: 11.4647 - acc: 0.0940
Epoch 2/5
1000/1000 [==============================] - 0s 39us/sample - loss: 11.4645 - acc: 0.0910
Epoch 3/5
1000/1000 [==============================] - 0s 42us/sample - loss: 11.4649 - acc: 0.0910
Epoch 4/5
1000/1000 [==============================] - 0s 37us/sample - loss: 11.4644 - acc: 0.0920
Epoch 5/5
1000/1000 [==============================] - 0s 40us/sample - loss: 11.4646 - acc: 0.0940


## 콜백

콜백(callback)은 훈련하는 동안 모델의 동작을 변경하고 확장하기 위해 전달하는 객체

자신만의 콜백을 작성하거나 다음과 같은 내장 `tf.keras.callbacks`을 사용할 수 있음:

* `tf.keras.callbacks.ModelCheckpoint`: 일정 간격으로 모델의 체크포인트를 저장합니다.
* `tf.keras.callbacks.LearningRateScheduler`: 학습률(learning rate)을 동적으로 변경합니다.
* `tf.keras.callbacks.EarlyStopping`: 검증 성능이 향상되지 않으면 훈련을 중지합니다.
* `tf.keras.callbacks.TensorBoard`: 텐서보드를 사용하여 모델을 모니터링합니다.


tf.keras.callbacks.Callback을 사용하려면 모델의 fit 메서드에 전달

In [15]:
callbacks = [
  # `val_loss`가 2번의 에포크에 걸쳐 향상되지 않으면 훈련을 멈춥니다.
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # `./logs` 디렉토리에 텐서보드 로그를 기록니다.
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 117us/sample - loss: 11.4642 - acc: 0.0950 - val_loss: 11.6625 - val_acc: 0.0400
Epoch 2/5
1000/1000 [==============================] - 0s 45us/sample - loss: 11.4643 - acc: 0.0940 - val_loss: 11.6629 - val_acc: 0.0400
Epoch 3/5
1000/1000 [==============================] - 0s 44us/sample - loss: 11.4649 - acc: 0.0920 - val_loss: 11.6632 - val_acc: 0.0400
